In [20]:
from langchain_core.runnables import ConfigurableField
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
from langchain_core.messages import HumanMessage, SystemMessage

In [21]:
# Load environment variables from .env file
load_dotenv()

# Retrieve the API key
api_key = os.getenv("OPENAI_API_KEY")

In [22]:
# Define tools using concise function definitions
@tool
def multiply(x: float, y: float) -> float:
   """Multiply 'x' times 'y'."""
   return x * y
 
@tool
def exponentiate(x: float, y: float) -> float:
   """Raise 'x' to the 'y'."""
   return x**y
 
@tool
def add(x: float, y: float) -> float:
   """Add 'x' and 'y'."""
   return x + y

In [23]:
tools = [multiply, exponentiate, add]
 
# Initialize the LLM with GPT-4o and bind the tools
llm = ChatOpenAI(model_name="gpt-4o", temperature=0, api_key=api_key)
llm_with_tools = llm.bind_tools(tools)

In [25]:
query = "What is 393 * 12.25? Also, what is 11 + 49?"
messages = [HumanMessage(query)]

In [26]:
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

In [27]:
messages

[HumanMessage(content='What is 393 * 12.25? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_l9PQ2GiE8JDiZddAdtjS2Reu', 'function': {'arguments': '{"x": 393, "y": 12.25}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_PCaSAHyZCja6uR1xkH23whdF', 'function': {'arguments': '{"x": 11, "y": 49}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 120, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-77517e8d-b8bc-4896-994d-96e76beb6bb2-0', tool_calls=[{'name': 'multiply', 'args': {'x': 393, 'y': 12.25}, 'id'

In [30]:
for tool_call in ai_msg.tool_calls:
   selected_tool = {"add": add, "multiply": multiply, "exponentiate": exponentiate}[tool_call["name"].lower()]
   tool_msg = selected_tool.invoke(tool_call)
  
   print(f"{tool_msg.name} {tool_call['args']} {tool_msg.content}")
   messages.append(tool_msg)

multiply {'x': 393, 'y': 12.25} 4814.25
add {'x': 11, 'y': 49} 60.0


In [31]:
final_response = llm_with_tools.invoke(messages)
print(final_response.content)

The result of \(393 \times 12.25\) is 4814.25, and the result of \(11 + 49\) is 60.
